# 0. Initialize

## 0.1. Import Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

## 0.2. DEFINE VARIABLES 

In [2]:
DATA_PATH = './datasets/' # '<insert-your-training-data-path-here>'

ROUND = 1 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '27041'#'<insert-your-id-here>'
PROJECT_CODE = 'CS412bee854a767f5'#'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx

## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [3]:
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
trainingTweetDf

,tweet_id,isPolitical
0,1597170281545551872,Yes
1,1431700027471192069,No
2,1566035577090281472,Yes
3,1591538690869940225,Yes
4,1583898169238167554,Yes
...,...,...
2995,1593539327623151619,Yes
2996,1393886554062524418,No
2997,1597925615092764672,Yes
2998,1585291418616176640,Yes


In [4]:
trainingTweetDf.isPolitical.value_counts()

Yes    2003
No      997
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [5]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
trainingUserDf

,screen_name,isBot
0,koftecancaddy,No
1,ahaber,No
2,selahat03949652,No
3,erdin06357062,No
4,bhct__necatii,No
...,...,...
2995,djblumenberg,No
2996,mel1sq,No
2997,eren_yz1,Yes
2998,ergnyildiz4,No


In [6]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [7]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS412bee854a767f5.html


# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [8]:
PATH_TO_DOWNLOADED = DATA_PATH # 'D:/Users/suuser/Desktop/Sabancı/CS412/spring-2022/project/'

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

#### 1.1.1.0 Check tweet_metadata's available features

In [9]:
with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    obj = json.loads(f.readline())
    print(obj.keys())
    print(obj['user'].keys())
    print(obj['entities'].keys())


dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])
dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'description', 'url', 'entities', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count', 'lang', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color'

### 1.1.1. Get Raw Data

#### 1.1.1.1. Check if Retweet

In [10]:
def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status']
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

#### 1.1.1.2. Get Tweet Text

In [11]:
def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text']
    
    return text

#### 1.1.1.3. Get Tweet ID

In [12]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

#### 1.1.1.4. Get Number of Mentions and Hashtags

In [13]:
def get_number_mentions_hashtags(tweet_metadata_line):
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])

    return num_mentions, num_hashtags

#### 1.1.1.5. Get Number of Retweets and Favorites

In [14]:
def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

#### 1.1.1.6. Get User Info

In [15]:
def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']

    return id, screen_name, description

### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Check for political entity in text

In [16]:
def check_political_ent(text):
    
    # the list below can be modified and some new names may be added (or removed)
    list_of_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 
                        'RTErdogan', 'MevlutCavusoglu', 'umitozdag', 
                        'Akparti', 'akpartiistanbul', 'herkesicinCHP', 
                        'iyiparti','akpartyenglish', 'yenisafak', 
                        'ekrem_imamoglu', 'kasapoglu',
                        'drfahrettinkoca','06melihgokcek',
                        'dbdevletbahceli','suleymansoylu', 'cenginyurt52', 
                        'DpGultekinUysal', 'MKursadSahin', 'alibabacan',
                        'mansuryavas06', 'ziyaselcuk',
                        'ahmet_davutoglu', 'ikalin1', 'numankurtulmus', 
                        'fahrettinaltun', 'omercelik', 'mhp_bilgi', 'fuatoktay',
                        'bbpgenelmerkez', 'mustafa_destici', 'vedatbilgin', 'dbdevletbahceli',
                        'arzuerdemdb', 'saffetsancakli', 'bybekirbozdag', 'mahirozdag']
    
    entities_in_text = [ent for ent in list_of_entities if ent.lower() in text.lower()]
    number_entities = len(entities_in_text)

    return number_entities

In [17]:
def check_political_words(text):
    
    list_of_words= [ 'chp','hdp','mhp', 'politika',
                    'akp','bakan', 'devlet',
                    'rte','tayyip','milletvekili', 'binali',
                    'iyi parti', 'akparti', 'genel başkan',
                    'rte', 'recep', 'tayyip', 'erdogan', 'başkan',
                    'parti', 'kürt', 'mafya', 'seçim', 'demokrasi',
                    'yerleştirme', 'demokrasi', 'erken',
                    'atama', 'millet', 'vatan', 'meral', 'bahçeli',
                    'cumhurbaşkanı', 'gençlik kolları', 'reis', 'yalan',
                    'iftira', 'altılı masa', 'koalisyon', 'baraj', 'suriye',
                    'cumhuriyet', 'hain', 'politik', 'politika', 'milli',
                    'aday', 'halk', 'düşman', 'sosyal', 'yurtsever',
                    'egemenlik', 'türkiye', 'türk', 'mülteci', 'savaş',
                    "Türkiye Cumhuriyeti", "Anayasa", "hükümet", "siyaset",
                    "seçim", "oy kullanma", "meclis", "başbakan", 
                    "cumhurbaşkanı", "milletvekili", "parti", "politika", 
                    "ideoloji", "seçim kampanyası", "vatandaşlık", "vergi",
                    "bütçe", "ekonomi", "politika", "milliyet", 
                    "kamu", "insan hakları", "sivil haklar", "ifade özgürlüğü",
                    "basın özgürlüğü", "protesto", "lobi", "rüşvet",
                    "yolsuzluk", "iktidar", "yargı", "federalizm", "güç ayrılığı", 
                    "denetim ve denge", "hukukun üstünlüğü", "seçim","temsil", 
                    "milletvekili", "kongre", "adaylık", "genel seçim", 
                    "yeniden seçim", "anayasa değişikliği", "başkanlık sistemi", 
                    "koalisyon", "çoğunluk", "azınlık", "veto", "oybirliği",
                    "genel af", "muhalefet", "lider", "feto"
    ]
    
    entities_in_text = [ent for ent in list_of_words if ent.lower() in text.lower()]
    number_words= len(entities_in_text)

    return number_words

#### 1.1.2.2. Number of total interactions

In [18]:
def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions

### 1.1.2. Collect data using the functions above and transform into a Pandas DataFrame

In [19]:
dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'user_description':[],
              'num_political_entities':[],
              'num_political_words':[],
              'num_political_words_in_user_description':[],
              'total_interactions':[]}


with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
        
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        text = get_tweet_text(line)
        num_mentions, num_hashtags = get_number_mentions_hashtags(line)
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description = get_user_info(line)

        # manually crafted data:
        num_political_entities = check_political_ent(text)
        num_political_words = check_political_words(text)
        num_political_words_in_user_description = check_political_words(line['user']['description'])
        total_num_interactions = total_interactions(retweet_count, favorite_count)

        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['user_description'].append(user_description)
        dfPolitical['num_political_entities'].append(num_political_entities)
        dfPolitical['num_political_words'].append(num_political_words)
        dfPolitical['num_political_words_in_user_description'].append(num_political_words_in_user_description)
        dfPolitical['total_interactions'].append(total_num_interactions)

In [20]:
dfPolitical = pd.DataFrame(dfPolitical)
dfPolitical

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,num_political_words,num_political_words_in_user_description,total_interactions
0,1588568792984346624,0,None,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,1,0,147
1,1588452263047069697,0,None,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,,0,1,0,0
2,1569589330544398336,0,None,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,2,1,0
3,1570428119609139201,0,None,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,0,0
4,1551163840368414722,0,None,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33528,1568595408233832448,0,None,Gerçek kimlik taşımayan hesaplara cevap vermem...,0,0,9,81,576247173,ardanzenturk,RT ONAYLADIĞIM ANLAMINA GELMEZ\nArtık fikirler...,0,0,0,90
33529,1584027427696959488,0,None,@umitozdag Neden Suriyelilerle ilgili bu kadar...,1,0,1,8,162308585,ozgul_61,Bridge design engineer Yaay hesabı : dilfiruz,1,2,0,9
33530,1585945783307730945,0,None,@celebimehmeta Niye Türkiye yüzyılıda.Türkiye ...,1,0,0,1,415025519,ladrekova,,0,4,0,1
33531,1569748909521801221,1,muazzezeralp,RT @muazzezeralp: @Doan58213655 @denizkonur @N...,7,1,6,0,1442125177727307781,yapikytgrivrlsn,,2,3,0,6


## 1.2. From Users

### 1.2.1. Get user metadata from user_profiles.jsons.gz

In [21]:
'''
Usefull Features:
In_reply
Accoung Age
Average In Reply
Average Favorites Count
Average Retweet Count
Average Total User Mention
days_std
Has Default Profile Picture
descbot
fav_count
favourites_count
Followers to Friends Ratuo
Followers Count
Friends Count
Has Extended Profile
hours_std
listedcount
name_length
nameratio
null_url
retweet_count
screennamebot
sn_length
total_usrmention
tweets_per_day
verified
'''

'\nUsefull Features:\nIn_reply\nAccoung Age\nAverage In Reply\nAverage Favorites Count\nAverage Retweet Count\nAverage Total User Mention\ndays_std\nHas Default Profile Picture\ndescbot\nfav_count\nfavourites_count\nFollowers to Friends Ratuo\nFollowers Count\nFriends Count\nHas Extended Profile\nhours_std\nlistedcount\nname_length\nnameratio\nnull_url\nretweet_count\nscreennamebot\nsn_length\ntotal_usrmention\ntweets_per_day\nverified\n'

#### 1.1.1.0 Check user_profiles's available features

In [22]:
with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    obj = json.loads(f.readline())
    print(obj.keys())
    print(obj['entities'].keys())

dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'description', 'url', 'entities', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count', 'lang', 'status', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_image_url', 'profile_image_url_https', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'has_extended_profile', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications', 'translator_type', 'withheld_in_countries'])
dict_keys(['description'])


#### 1.2.1.1. Get user info metadata

In [23]:
def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    user_favorites_count= user_metadata_line['favourites_count']
    user_statuses_count= user_metadata_line['statuses_count']
    user_protected= user_metadata_line['protected']
    user_verified = user_metadata_line['verified']
    user_has_url = user_metadata_line['url'] is not None
    user_has_default_photo = user_metadata_line['default_profile_image']
    user_has_extended_profile = user_metadata_line['has_extended_profile']


    dictionary = {
        'user_id':user_id,
        'user_name': user_name,
        'user_screen_name':user_screen_name,
        'user_location':user_location,
        'user_description':user_description,
        'user_followers_count':user_followers_count,
        'user_favorites_count':user_favorites_count,
        'user_statuses_count':user_statuses_count,
        'user_friends_count':user_friends_count,
        'user_protected':user_protected,        
        'user_verified': user_verified,
        'user_has_url': user_has_url,
        'user_has_default_photo': user_has_default_photo,
        'user_has_extended_profile': user_has_extended_profile,
    }

    return dictionary

#### 1.2.1.2. Get followers/(followers+friends) ratio

In [24]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

#### 1.2.1.3. Get description length

In [25]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

In [26]:
dfBot = {'user_id':[],
        'user_name':[],
        'user_screen_name':[],
        'user_location':[],
        'user_description':[],
        'user_followers_count':[],
        'user_friends_count':[],
        'description_len':[],
        'user_verified':[],
        'followers_to_all_ratio':[],
        'user_favorites_count':[],
        'user_protected':[],
        'user_statuses_count':[],
        'user_has_url':[],
        'user_has_default_photo':[],
        'user_has_extended_profile':[]
        }

with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], dictionary['user_friends_count'])

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

In [27]:
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,user_verified,followers_to_all_ratio,user_favorites_count,user_protected,user_statuses_count,user_has_url,user_has_default_photo,user_has_extended_profile
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,False,0.260000,17676,False,2551,False,False,True
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,False,0.732260,15474,False,42771,True,False,True
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,False,0.192308,18220,False,14300,False,False,True
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,False,0.325203,26999,False,21303,False,False,False
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,False,0.505051,2179,False,1629,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,False,0.513453,10820,False,2396,False,False,False
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,False,0.975088,36671,False,75178,False,False,True
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,False,0.451362,7389,False,6482,False,False,False
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,False,0.314431,140095,False,121113,False,False,True


### 1.2.2. Get Tweet Info of Users in user_profiles.jsons.gz

#### 1.1.1.0 Check user_profiles's available features

In [28]:
with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    obj = json.loads(f.readline())
    print(obj.keys())
    print(obj['tweets'][0].keys())
    print(obj['tweets'][0]['entities'].keys())

dict_keys(['tweets', 'user_id'])
dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])
dict_keys(['hashtags', 'symbols', 'user_mentions', 'urls'])


#### 1.2.2.1. Check ratio of retweets to all tweets

In [29]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    
    return 0 if total_tweets == 0 else number_retweets/(total_tweets)

In [30]:
def in_reply_to_ratio(line):
    number_of_replies = 0
    number_of_tweet = 0

    for tweet in line['tweets']:
        if tweet['in_reply_to_screen_name'] is not None:
            number_of_replies += 1
        number_of_tweet += 1
    
    return 0 if number_of_tweet == 0 else number_of_replies/number_of_tweet

In [31]:
def get_average_mentions(line):
    number_of_mentions = 0
    total = 0
    for tweet in line['tweets']:
        for mention in tweet['entities']['user_mentions']:
            if tweet['in_reply_to_screen_name'] is not None:
                number_of_mentions += 1
            total += 1
    
    return 0 if number_of_mentions == 0 else number_of_mentions / total

#### 1.2.2.2. Check median number of favorites

In [32]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [33]:
dfBotTweets = { 'user_id':[],
                'retweet_total_ratio':[],
                'num_median_favorites':[],
                'num_of_tweets':[],
                'in_reply_to_ratio':[],
                'average_mentions':[]
                }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)

        reply_to_ratio = in_reply_to_ratio(line)
        dfBotTweets['in_reply_to_ratio'].append(reply_to_ratio)
        
        average_mentions = get_average_mentions(line)
        dfBotTweets['average_mentions'].append(average_mentions)

        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        dfBotTweets['num_of_tweets'].append(len(line['tweets']))

        i += 1
        if i % 1000 == 0:
            print(i)

/Users/emirkantul/Documents/Repos/bot-and-political-tweet-detection/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/emirkantul/Documents/Repos/bot-and-political-tweet-detection/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [34]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,retweet_total_ratio,num_median_favorites,num_of_tweets,in_reply_to_ratio,average_mentions
0,594642154,0.115000,2.0,200,0.530000,0.838926
1,525600289,0.005025,1.0,199,0.778894,0.930380
2,931895965501534209,0.900000,0.0,200,0.025000,0.010204
3,1591543462746329088,0.185000,0.0,200,0.795000,0.905983
4,734801354749796352,1.000000,0.0,200,0.000000,0.000000
...,...,...,...,...,...,...
28310,1591370361488252928,0.800000,0.0,200,0.120000,0.104895
28311,1475272459616235525,0.825000,0.0,200,0.155000,0.140777
28312,1096753792731750401,0.051020,1.0,196,0.632653,0.910526
28313,1269527617687953409,0.095000,2.0,200,0.155000,0.691176


### 1.2.3. Merge dfBot and dfBotTweets

In [35]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

dfBotAll[['retweet_total_ratio', 'num_median_favorites']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites']].fillna(0)

dfBotAll

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,user_verified,followers_to_all_ratio,user_favorites_count,user_protected,user_statuses_count,user_has_url,user_has_default_photo,user_has_extended_profile,retweet_total_ratio,num_median_favorites,num_of_tweets,in_reply_to_ratio,average_mentions
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,False,0.260000,17676,False,2551,False,False,True,0.395939,0.0,197.0,0.604061,0.594262
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,False,0.732260,15474,False,42771,True,False,True,0.125000,0.0,200.0,0.635000,0.787879
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,False,0.192308,18220,False,14300,False,False,True,0.910000,0.0,200.0,0.005000,0.001838
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,False,0.325203,26999,False,21303,False,False,False,0.015306,1.0,196.0,0.331633,0.946429
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,False,0.505051,2179,False,1629,False,False,False,0.659898,0.0,197.0,0.111675,0.153409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,False,0.513453,10820,False,2396,False,False,False,0.015000,1.0,200.0,0.940000,0.973094
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,False,0.975088,36671,False,75178,False,False,True,0.291457,2.0,199.0,0.311558,0.480263
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,False,0.451362,7389,False,6482,False,False,False,0.061538,0.0,195.0,0.938462,0.929348
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,False,0.314431,140095,False,121113,False,False,True,0.995000,0.0,200.0,0.005000,0.004587


# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

### 2.1.1. Merge dfPolitical data with labels

In [36]:
dfPoliticalAll_train = dfPolitical.merge(trainingTweetDf,
                                         on='tweet_id')

dfPoliticalAll_train.head()

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,num_political_words,num_political_words_in_user_description,total_interactions,isPolitical
0,1585955683513798656,0,None,@AvOzlemZengin YüzüncüYıla YakışanGenelAf adli...,1,0,3,2,1564992353168941058,zehra78231638,,0,1,0,5,Yes
1,1597631718479261696,0,None,#TCYüzyılıÜcretliÖgrtKadro\n#TCYüzyılıÜcretliÖ...,0,2,30,28,1324630334416297985,nurozguler,,0,0,0,58,Yes
2,1572522789948751874,0,None,Ekrem İmamoğlu davayı değerlendirdi. 'Boş işle...,0,0,5,66,407597071,onediocom,Türkiye'nin ilk ve tek sosyal içerik sitesi ht...,0,0,3,71,Yes
3,1591412481561624577,0,None,Sayın Bakanım @suleymansoylu POMEM önlisans er...,1,0,0,0,1394789887073738753,buckybarnestr,...,1,2,0,0,Yes
4,1596914274907348992,0,None,"@varank Sayın bakanım, Bodrumdaki bu araziyi ...",1,0,0,0,1586083256088371201,sayariahmet,,0,1,0,0,Yes


### 2.1.2. Separate X and y values
We only use 3 features here to create a baseline model. However, it is not enough to get good results.

In [37]:
X = dfPoliticalAll_train[['is_retweet','num_mentions','num_hashtags', 'num_retweets', 'num_favorites', 'num_political_entities', 'num_political_words', 'num_political_words_in_user_description', 'total_interactions']]
y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

### 2.1.3. Train - validation split

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.1.4. Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [39]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_political = DecisionTreeClassifier()

# fit your model
dtc_political.fit(X_train, y_train)

# make predictions
preds = dtc_political.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.26166666666666666 
 Accuracy Score: 0.7383333333333333 
 Confusion Matrix: 
 [[108 101]
 [ 56 335]]


In [40]:
# Random Forest
# create an instance
rfc_political = RandomForestClassifier()

# fit your model
rfc_political.fit(X_train, y_train)

# make predictions
preds = rfc_political.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)


MSE: 0.23833333333333334 
 Accuracy Score: 0.7616666666666667 
 Confusion Matrix: 
 [[101 108]
 [ 35 356]]


In [43]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
params = {
    'max_depth': range(3, 10, 2),
    "min_child_weight": [3, 5, 7],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'gamma':[i/10.0 for i in range(0,5)]
}
numnum = GridSearchCV(XGBClassifier(), params, cv=5,scoring = 'accuracy', 
                    return_train_score=False, 
                    verbose=1, 
                    refit=True,
                    n_jobs=-1)
numnum.fit(X_train, y_train)
numnum.best_params_

Fitting 5 folds for each of 960 candidates, totalling 4800 fits


KeyboardInterrupt: 

In [ ]:
numnum = XGBClassifier(learning_rate= 0.1, min_child_weight = 7, n_estimators= 550, colsample_bytree=0.6, gamma=0.0, max_depth=5, subsample=0.6)
numnum.fit(X_train, y_train)
preds = numnum.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)
best_politic_model = numnum

MSE: 0.24666666666666667 
 Accuracy Score: 0.7533333333333333 
 Confusion Matrix: 
 [[ 99 110]
 [ 38 353]]


In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

def train_bot_detection_model(X_train, y_train):
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Train the model
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train_scaled, y_train)
    
    return model, scaler

def predict_with_bot_detection_model(model, scaler, X_test):
    # Make predictions on the test set
    X_test_scaled = scaler.transform(X_test)
    y_pred = model.predict(X_test_scaled)
    return y_pred


## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [45]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [46]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')



In [47]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

In [48]:
X = dfBotAll_train[['retweet_total_ratio', 'num_median_favorites', 'in_reply_to_ratio', 'average_mentions', 'user_statuses_count', 'user_has_url', 'user_verified', 'user_protected', 'user_has_extended_profile', 'user_has_default_photo']]
y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

X = X.fillna(0) # fill na with 0


### 2.2.3. Train-test split

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.2.4. Train the model

In [50]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_bot = DecisionTreeClassifier()

# fit your model
dtc_bot.fit(X_train, y_train)

# make predictions
preds = dtc_bot.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.23833333333333334 
 Accuracy Score: 0.7633333333333333 
 Confusion Matrix: 
 [[420  73]
 [ 69  38]]


In [53]:
from sklearn.metrics import make_scorer

rf_clf2 = RandomForestClassifier()

parameters = {'n_estimators': [10,20,25], 
              'max_features': [10,12,16], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [6, 8, 10,12], 
              'min_samples_split': [20, 30, 50],
              'min_samples_leaf': [3,5,8]
             }

acc_scorer = make_scorer(accuracy_score)

#grid search
grid_obj = GridSearchCV(rf_clf2, parameters, scoring=acc_scorer, n_jobs=-1)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
rf_clf2 = grid_obj.best_estimator_

rf_clf2.fit(X_train, y_train)



RandomForestClassifier(criterion='entropy', max_depth=8, max_features=10,
                       min_samples_leaf=8, min_samples_split=30,
                       n_estimators=10)

In [54]:
from sklearn.ensemble import AdaBoostClassifier
ada2 = AdaBoostClassifier(base_estimator=rf_clf2, n_estimators=50)
ada2.fit(X_train, y_train)

/Users/emirkantul/Documents/Repos/bot-and-political-tweet-detection/.venv/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=RandomForestClassifier(criterion='entropy',
                                                         max_depth=8,
                                                         max_features=10,
                                                         min_samples_leaf=8,
                                                         min_samples_split=30,
                                                         n_estimators=10))

In [55]:
preds = ada2.predict(X_valid)

acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

best_bot_model = ada2

MSE: 0.23833333333333334 
 Accuracy Score: 0.7883333333333333 
 Confusion Matrix: 
 [[443  50]
 [ 77  30]]


# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [ ]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('{}evaluation-round2-tweet.csv'.format(DATA_PATH), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfPolitical_test[['is_retweet','num_mentions','num_hashtags', 'num_retweets', 'num_favorites', 'num_political_entities', 'num_political_words', 'num_political_words_in_user_description', 'total_interactions']]

# make predictions based on these variables
predictions_political = best_politic_model.predict(X)

### This part is important! We expect you to return your predictions in the following format:

In [ ]:
modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

{'1593649159009099777': 1.0,
 '1367571642604544000': 0.0,
 '1589993032975544320': 1.0,
 '1565312596135354373': 1.0,
 '1388235183653011462': 1.0,
 '1592120408073203712': 0.0,
 '1439547067337256967': 0.0,
 '1597274845381029888': 1.0,
 '1586021183958704128': 1.0,
 '1356926480605982728': 0.0,
 '1595357036925026306': 1.0,
 '1585766233491886081': 1.0,
 '1595871258985615361': 1.0,
 '1352635736537882629': 1.0,
 '1583477966373543936': 1.0,
 '1564926450096013313': 0.0,
 '1585634359612420101': 1.0,
 '1384499047390658560': 0.0,
 '1596583748669419521': 1.0,
 '1391681495622995971': 0.0,
 '1365710259549966339': 0.0,
 '1590673118397624323': 1.0,
 '1389951943343316995': 0.0,
 '1407921226656280580': 0.0,
 '1452348722810138646': 0.0,
 '1597256187325878273': 1.0,
 '1595829502021623812': 1.0,
 '1579408398894137344': 1.0,
 '1570758749606019073': 1.0,
 '1366091745772077058': 0.0,
 '1596233602886701057': 1.0,
 '1584922292127256577': 1.0,
 '1586279180983042050': 1.0,
 '1399687111234756612': 0.0,
 '136322656468

## 3.2. Predictions for Users (Bot or Not)

In [429]:
evaluationUserDf = pd.read_csv('{}evaluation-round2-user.csv'.format(DATA_PATH), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBot_test[['retweet_total_ratio', 'num_median_favorites', 'in_reply_to_ratio', 'total_user_mentions', 'user_statuses_count', 'user_has_url', 'user_verified', 'user_protected', 'user_has_extended_profile', 'user_has_default_photo']]

# make predictions based on these variables
predictions_bot = best_bot_model.predict(X)

In [430]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'nedenburdaysam': 0.0,
 'biologselim': 0.0,
 'alaraaynncnm': 0.0,
 '_sydneycarton_': 0.0,
 'denizlihabercom': 0.0,
 'burakerbaychp': 0.0,
 'mustafaarst': 0.0,
 'mvnez': 0.0,
 'qara118': 0.0,
 'alpar_kaan': 0.0,
 'farukhalit2': 0.0,
 'haf_zhan': 0.0,
 'harlunoshi': 0.0,
 'heritagepaix': 0.0,
 '37baho37': 0.0,
 'tamerduran_1': 0.0,
 'donkisotumsu': 0.0,
 'nuranwolf': 0.0,
 'politikgundem': 0.0,
 'isakethudax': 0.0,
 'ilaydejaneiro': 0.0,
 'gendenmukatol': 0.0,
 '1905anason': 1.0,
 'eraydurgut03': 0.0,
 'dasiskein': 0.0,
 'mett_1907': 0.0,
 'semihyeteer': 0.0,
 'haberinyokcokk': 0.0,
 'meleky_ozaydin': 0.0,
 'han34nesli': 0.0,
 'bilobi4': 0.0,
 'berkeduranovic': 0.0,
 'cagdasadim': 0.0,
 'merabalare': 0.0,
 'sevdaac72373936': 0.0,
 '21gramlife1': 0.0,
 'cakan0_': 0.0,
 'oguzksalici': 0.0,
 'emre_caliskann': 0.0,
 'mehmet07454846': 0.0,
 'lifegs': 0.0,
 'begumkarabeyx': 0.0,
 'avutulan': 0.0,
 'imamgibiimam': 0.0,
 'durdane52': 0.0,
 'radyotrafik35': 0.0,
 'rk_ozanyali': 0.0,
 'uykusuz75'

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [431]:
# Explain your approach

data_explanations = 'I have used to given data set to analyze with mine as well. Also I did research and what previously done and what results they got, which model and features they used. Also I found our professors BotOrNot research and implementation to get idea. I didn\' t have time to test which features to use or their accuracies but on further rounds I will do proper testing'''

feature_explanations = '''
First I checked what features are available and select them accordingly. also derived some features myself like number of political words in description etc.
'''

model_explanations = '''
I have tested and researched different models and decided on random forest as it gives the best results. Also I want to add AdaBoosting to my tree models and test them on next round
'''

additional_explanations = '''
A proper research on project and what people done previously was very helpful and on future rounds I have some more ideas as well
'''


In [432]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

In [433]:
# Test your submission file

submission = json.load(open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'r'))
submission

{'round': 1,
 'student_id': '27041',
 'user_predictions': {'nedenburdaysam': 0.0,
  'biologselim': 0.0,
  'alaraaynncnm': 0.0,
  '_sydneycarton_': 0.0,
  'denizlihabercom': 0.0,
  'burakerbaychp': 0.0,
  'mustafaarst': 0.0,
  'mvnez': 0.0,
  'qara118': 0.0,
  'alpar_kaan': 0.0,
  'farukhalit2': 0.0,
  'haf_zhan': 0.0,
  'harlunoshi': 0.0,
  'heritagepaix': 0.0,
  '37baho37': 0.0,
  'tamerduran_1': 0.0,
  'donkisotumsu': 0.0,
  'nuranwolf': 0.0,
  'politikgundem': 0.0,
  'isakethudax': 0.0,
  'ilaydejaneiro': 0.0,
  'gendenmukatol': 0.0,
  '1905anason': 1.0,
  'eraydurgut03': 0.0,
  'dasiskein': 0.0,
  'mett_1907': 0.0,
  'semihyeteer': 0.0,
  'haberinyokcokk': 0.0,
  'meleky_ozaydin': 0.0,
  'han34nesli': 0.0,
  'bilobi4': 0.0,
  'berkeduranovic': 0.0,
  'cagdasadim': 0.0,
  'merabalare': 0.0,
  'sevdaac72373936': 0.0,
  '21gramlife1': 0.0,
  'cakan0_': 0.0,
  'oguzksalici': 0.0,
  'emre_caliskann': 0.0,
  'mehmet07454846': 0.0,
  'lifegs': 0.0,
  'begumkarabeyx': 0.0,
  'avutulan': 0.